ANSI Color Overrides
==============================================================================

Script to get some usable dark-mode ANSI colors from my iterm config, See
[dev/notes/2025-11-28.rich-theme.ipynb](./2025-11-28.rich-theme.ipynb) for
why we need ANSI colors.

In [8]:
from collections.abc import Iterable
import plistlib
from pathlib import Path
import rich
from rich.text import Text
from collections.abc import Mapping
import re

NAMES = [
    n.lower().replace(" ", "_")
    for n in [
        "Black",
        "Red",
        "Green",
        "Yellow",
        "Blue",
        "Magenta",
        "Cyan",
        "White",
        "Bright Black",
        "Bright Red",
        "Bright Green",
        "Bright Yellow",
        "Bright Blue",
        "Bright Magenta",
        "Bright Cyan",
        "Bright White",
    ]
]


def _clamp_to_byte(x: float) -> int:
    """Convert a float in [0, 1] to an 8-bit integer [0, 255]."""
    if x is None:
        raise ValueError("component is None; expected float in [0, 1]")
    return max(0, min(255, round(float(x) * 255)))


def rgb_floats_to_hex(rgb: Iterable[float], alpha: float | None = None) -> str:
    """
    Convert RGB floats (0.0–1.0) to hex color.

    Examples:
        rgb_floats_to_hex((0.313, 0.614, 0.918)) -> '#509dea'
        rgb_floats_to_hex((0.313, 0.614, 0.918), 0.8) -> '#509deaCC'
    """
    r_f, g_f, b_f = tuple(rgb)[:3]
    r, g, b = (_clamp_to_byte(c) for c in (r_f, g_f, b_f))
    if alpha is None:
        return f"#{r:02x}{g:02x}{b:02x}"
    a = _clamp_to_byte(alpha)
    return f"#{r:02x}{g:02x}{b:02x}{a:02x}"


def iterm_color_dict_to_hex(color: Mapping[str, float]) -> str:
    """
    Convert an iTerm .itermcolors-style color mapping to hex.

    Expects keys like 'Red Component', 'Green Component', 'Blue Component'.
    """
    r = color.get("Red Component")
    g = color.get("Green Component")
    b = color.get("Blue Component")
    if r is None or g is None or b is None:
        missing = [
            k
            for k, v in (
                ("Red Component", r),
                ("Green Component", g),
                ("Blue Component", b),
            )
            if v is None
        ]
        raise KeyError(f"missing components: {', '.join(missing)}")
    return rgb_floats_to_hex((r, g, b))


with (
    Path("~/.config/iterm2/nrser.itermcolors")
    .expanduser()
    .resolve()
    .open("rb") as f
):
    iterm_colors = plistlib.load(f)

hex_colors = {}

for name, color in iterm_colors.items():
    if m := re.match(r"^Ansi\ (\d+)\ Color", name):
        i = int(m.group(1))
        name = NAMES[i]
        hex = iterm_color_dict_to_hex(color)
        hex_colors[name] = hex

rich.print(*(Text(f"{n}: {c}\n", c) for n, c in hex_colors.items()))

black: #0e0f12
 red: #e06c75
 bright_green: #5cb85c
 bright_yellow: #f0ad4e
 bright_blue: #52acf7
 bright_magenta: #b95bde
 bright_cyan: #5bc0de
 bright_white: #ffffff
 green: #98c379
 yellow: #e5c07b
 blue: #61afef
 magenta: #c678dd
 cyan: #56b6c2
 white: #dee1de
 bright_black: #636a80
 bright_red: #d9534f

Print the colors out in keyword-argument form.

In [12]:
print("\n".join([f'{n}="{c}",' for n, c in hex_colors.items()]))

black="#0e0f12",
red="#e06c75",
bright_green="#5cb85c",
bright_yellow="#f0ad4e",
bright_blue="#52acf7",
bright_magenta="#b95bde",
bright_cyan="#5bc0de",
bright_white="#ffffff",
green="#98c379",
yellow="#e5c07b",
blue="#61afef",
magenta="#c678dd",
cyan="#56b6c2",
white="#dee1de",
bright_black="#636a80",
bright_red="#d9534f",
